In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

# Data Science libraries
import pandas as pd

import time
from requests import get

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


In [2]:
API = 'https://api.github.com/'

In [3]:
import requests
def get_page(url, headers=headers):
    response = requests.get(url, headers=headers)
    return response.json()

In [4]:
response = requests.get(API, headers=headers).json()
response['rate_limit_url']

'https://api.github.com/rate_limit'

In [5]:
{'repository_search_url': 'https://api.github.com/search/repositories?q={query}{&page,per_page,sort,order}',
 'topic_search_url': 'https://api.github.com/search/topics?q={query}{&page,per_page}',
 'repository_url': 'https://api.github.com/repos/{owner}/{repo}',
  'rate_limit_url': 'https://api.github.com/rate_limit',
}

{'repository_search_url': 'https://api.github.com/search/repositories?q={query}{&page,per_page,sort,order}',
 'topic_search_url': 'https://api.github.com/search/topics?q={query}{&page,per_page}',
 'repository_url': 'https://api.github.com/repos/{owner}/{repo}',
 'rate_limit_url': 'https://api.github.com/rate_limit'}

In [6]:
requests.get(response['rate_limit_url'], headers=headers).json()

{'resources': {'core': {'limit': 5000,
   'used': 3861,
   'remaining': 1139,
   'reset': 1662164289},
  'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1662163212},
  'graphql': {'limit': 5000,
   'used': 0,
   'remaining': 5000,
   'reset': 1662166752},
  'integration_manifest': {'limit': 5000,
   'used': 0,
   'remaining': 5000,
   'reset': 1662166752},
  'source_import': {'limit': 100,
   'used': 0,
   'remaining': 100,
   'reset': 1662163212},
  'code_scanning_upload': {'limit': 1000,
   'used': 0,
   'remaining': 1000,
   'reset': 1662166752},
  'actions_runner_registration': {'limit': 10000,
   'used': 0,
   'remaining': 10000,
   'reset': 1662166752},
  'scim': {'limit': 15000, 'used': 0, 'remaining': 15000, 'reset': 1662166752},
  'dependency_snapshots': {'limit': 100,
   'used': 0,
   'remaining': 100,
   'reset': 1662163212}},
 'rate': {'limit': 5000, 'used': 3861, 'remaining': 1139, 'reset': 1662164289}}

In [7]:
base_url = 'https://api.github.com/'
query = 'stars%3A%3E1&s=forks&type=Repositories'
page = 1
url = f"{base_url}search/repositories?q={query}&page={page}"
response = requests.get(url, headers=headers)

In [8]:
for k, v in response.headers.items():
    print(f"{k}: {v}")

Server: GitHub.com
Date: Fri, 02 Sep 2022 23:59:13 GMT
Content-Type: application/json; charset=utf-8
Transfer-Encoding: chunked
Cache-Control: no-cache
Vary: Accept, Authorization, Cookie, X-GitHub-OTP, Accept-Encoding, Accept, X-Requested-With
X-OAuth-Scopes: 
X-Accepted-OAuth-Scopes: 
github-authentication-token-expiration: 2022-10-02 15:09:16 UTC
X-GitHub-Media-Type: github.v3; format=json
Link: <https://api.github.com/search/repositories?q=stars%3A%3E1&s=forks&type=Repositories&page=2>; rel="next", <https://api.github.com/search/repositories?q=stars%3A%3E1&s=forks&type=Repositories&page=34>; rel="last"
X-RateLimit-Limit: 30
X-RateLimit-Remaining: 29
X-RateLimit-Reset: 1662163213
X-RateLimit-Used: 1
X-RateLimit-Resource: search
Access-Control-Expose-Headers: ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type,

In [9]:
response.headers['X-RateLimit-Remaining'], response.status_code

('29', 200)

In [10]:
def github_search(query, page):
    # topic_search_url': 'https://api.github.com/search/topics?q={query}{&page,per_page}',
    base_url = 'https://api.github.com/'
    url = f"{base_url}search/repositories?q={query}&page={page}&per_page=100"
    time.sleep(2)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f'Something went wrong. Status {response.status_code}')
    else:
        return response.json()

In [11]:
from ipywidgets import IntProgress
import time

max_count = 100

f = IntProgress(min=0, max=max_count, description='Loading') # instantiate the bar
display(f) # display the bar

count = 0
while count <= max_count:
    f.value += 1 # signal to increment the progress bar
    time.sleep(.01)
    count += 1

IntProgress(value=0, description='Loading')

In [12]:
def get_github_urls():
    from ipywidgets import IntProgress
    urls = []
    page = 0
    f = IntProgress(min=0, max=1000, description='Getting URLs')
    display(f)
    while len(urls) <+ 900:
        f.value = len(urls)
        page += 1
        items = github_search('stars%3A%3E1&s=forks&type=Repositories', page)['items']
        for item in items:
            urls.append(item['full_name'])
    return urls


In [14]:
urls = get_github_urls()

IntProgress(value=0, description='Getting URLs', max=1000)

In [15]:
df = pd.DataFrame(urls, columns=['repos'])
df.repos.nunique(), df.shape

(1000, (1000, 1))

In [16]:
df.to_csv('git_urls.csv', index=False)

In [ ]:
github_search('stars%3A%3E1&s=forks&type=Repositories', 35)

In [18]:
import acquire

In [19]:
df = acquire.wrangle_data()

In [ ]:
df.repo.nunique()

804

In [20]:
import pandas as pd
xs = pd.DataFrame(acquire.REPOS, columns=['repos'])
xs.repos.nunique()

1000

In [21]:
repos = acquire.REPOS

df = df.drop_duplicates()

In [27]:
skip = df.repo.tolist()

In [29]:
to_get = [repo for repo in repos if repo not in skip]

In [30]:
len(to_get)

227

In [31]:
acquire.REPOS = to_get

In [33]:
temp = acquire.wrangle_data()

IntProgress(value=0, description='Downloading Data', max=227)

https://api.github.com/repos/sindresorhus/awesome/contents/
https://api.github.com/repos/sindresorhus/awesome
https://api.github.com/repos/donnemartin/system-design-primer/contents/
https://api.github.com/repos/donnemartin/system-design-primer
https://api.github.com/repos/codecrafters-io/build-your-own-x/contents/
https://api.github.com/repos/codecrafters-io/build-your-own-x
https://api.github.com/repos/nodejs/node/contents/
https://api.github.com/repos/nodejs/node
https://api.github.com/repos/mrdoob/three.js/contents/
https://api.github.com/repos/mrdoob/three.js
https://api.github.com/repos/goldbergyoni/nodebestpractices/contents/
https://api.github.com/repos/goldbergyoni/nodebestpractices
https://api.github.com/repos/puppeteer/puppeteer/contents/
https://api.github.com/repos/puppeteer/puppeteer
https://api.github.com/repos/iluwatar/java-design-patterns/contents/
https://api.github.com/repos/iluwatar/java-design-patterns
https://api.github.com/repos/nvbn/thefuck/contents/
https://api.

In [34]:
temp.shape

(227, 3)

In [35]:
pd.concat([df, temp]).shape

(1031, 3)

In [36]:
final = pd.concat([df, temp])

In [39]:
final.drop_duplicates().to_csv('data.csv', index=False)